In [29]:
import torchaudio
import torch
from pathlib import Path
from datasets import concatenate_datasets

In [2]:

def split_wav(file_path, out_dir, chunk_duration=0.5, target_sr=16000):
    """
    Split a WAV file into fixed-length chunks and save them.

    Parameters:
        file_path (str or Path): Path to input WAV file.
        out_dir (str or Path): Directory to save chunks.
        chunk_duration (float): Duration of each chunk in seconds (default 0.5s).
        target_sr (int): Target sampling rate (default 16000 Hz).
    """
    file_path = Path(file_path)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load audio
    waveform, sr = torchaudio.load(file_path)

    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample if needed
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(sr, target_sr)
        waveform = resampler(waveform)
        sr = target_sr

    num_samples_per_chunk = int(chunk_duration * sr)
    total_samples = waveform.shape[1]
    num_chunks = (total_samples + num_samples_per_chunk - 1) // num_samples_per_chunk

    # Split and save
    for i in range(num_chunks):
        start = i * num_samples_per_chunk
        end = min(start + num_samples_per_chunk, total_samples)
        chunk = waveform[:, start:end]

        # Pad last chunk if too short
        if chunk.shape[1] < num_samples_per_chunk:
            padding = num_samples_per_chunk - chunk.shape[1]
            chunk = torch.nn.functional.pad(chunk, (0, padding))

        out_file = out_dir / f"{file_path.stem}_chunk{i+1}.wav"
        torchaudio.save(str(out_file), chunk, sr)

    print(f"✅ Saved {num_chunks} chunks to {out_dir}")


In [8]:
# split_wav("/home/pierre/Downloads/drone-sound-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/bb")
# split_wav("/home/pierre/Downloads/this-is-what-a-drone-sounds-like-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
# split_wav("/home/pierre/Downloads/this-drone-flight-is-unreal-128-ytshorts.savetube.me.wav", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
# split_wav("/home/pierre/Downloads/drone-sound-effects-128-ytshorts.savetube.me.mp3", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")
# split_wav("/home/pierre/Downloads/chilling-sound-of-a-uaf-drone-hunting-for-a-cowering-russian-soldier-128-ytshorts.savetube.me.mp3", "/home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa")

✅ Saved 30 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/test/aa


In [3]:
import os

def process_wav_folder(input_dir, output_dir):
    """
    Processes all .wav files in a given folder using split_wav(file_path, out_dir).

    Args:
        input_dir (str): Path to the folder containing .wav files.
        output_dir (str): Path to the folder where processed files will be saved.
    """

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".wav"):
            file_path = os.path.join(input_dir, filename)
            print(f"Processing: {file_path}")

            try:
                split_wav(file_path, output_dir)
            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")

    print("✅ All .wav files processed successfully!")



In [6]:
# process_wav_folder("/home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test", "/home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test_output")

Processing: /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test/45sec_dji-avata-sound-243279.wav
✅ Saved 92 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test_output
Processing: /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test/002383_drone-parrot-mambomp3-52899_2.wav
✅ Saved 754 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test_output
Processing: /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test/039493_drone-electronic-pain-71147_2.wav
✅ Saved 47 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test_output
Processing: /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test/a-dji-mini-3-drone-from-its-pre-flight-checks-to-a-smooth-landing-393572_2.wav
✅ Saved 41 chunks to /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test_output
Processing: /home/pierre/Documents/Projects/PST4/AI/data/raw/new/drone_test/dji-avata2-370956_2.wav
✅ Saved 4 chunks to /home/pierre/

In [30]:
from datasets import load_dataset, ClassLabel, DatasetDict, concatenate_datasets

ds = load_dataset("audiofolder", data_dir="../data/raw/new/new_test_ds")
ds = DatasetDict({"test": ds["train"]})

In [18]:
labels = ds["test"].features["label"]

In [33]:
file_names = [p.split("/")[-1] for p in ds["test"].__dict__["_info"].download_checksums.keys()]
ds["test"] = ds["test"].add_column("filename", file_names)

In [34]:
ds

DatasetDict({
    test: Dataset({
        features: ['audio', 'label', 'filename'],
        num_rows: 2805
    })
})

In [25]:
from IPython.display import display, Audio
ad1 = ds["test"][2084]
print(labels.int2str(ad1["label"]), ad1["filename"])
display(Audio(ad1["audio"]["array"], rate=ad1["audio"]["sampling_rate"]))

drone sound-of-flying-drone_02-166962_2_chunk351.wav


In [26]:
old_dataset = load_dataset("Usernameeeeee/drone_test")

In [35]:
print(ds)
print(old_dataset)

DatasetDict({
    test: Dataset({
        features: ['audio', 'label', 'filename'],
        num_rows: 2805
    })
})
DatasetDict({
    test: Dataset({
        features: ['audio', 'label', 'filename'],
        num_rows: 893
    })
})


In [37]:
new_dataset = concatenate_datasets([ds["test"], old_dataset["test"]])

In [39]:
ds.push_to_hub("Usernameeeeee/drone_test_2")

Map: 100%|██████████| 2805/2805 [00:00<00:00, 5255.37 examples/s]

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  4.84ba/s]
Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (0 / 1):   1%|          |  566kB / 74.1MB,  707kB/s  
Processing Files (0 / 1):   1%|▏         | 1.11MB / 74.1MB,  789kB/s  
Processing Files (0 / 1):   3%|▎         | 2.21MB / 74.1MB,  921kB/s  
Processing Files (0 / 1):   4%|▍         | 3.32MB / 74.1MB,  872kB/s  
Processing Files (0 / 1):   5%|▌         | 3.86MB / 74.1MB,  803kB/s  
Processing Files (0 / 1):   6%|▌         | 4.42MB / 74.1MB,  884kB/s  
Processing Files (0 / 1):   7%|▋         | 4.96MB / 74.1MB,  800kB/s  
Processing Files (0 / 1):   7%|▋         | 5.53MB / 74.1MB,  863kB/s  
Processing Files (0 / 1):   8%|▊         | 6.07MB / 74.1MB,  892kB/s  
Processing Files (0 / 1):  10%|▉         | 7.17MB / 74.1MB,  943kB/s  
Processing Files (0 / 1):  10%|█         | 7.74MB / 74.1MB,  943kB/s  
Pro

CommitInfo(commit_url='https://huggingface.co/datasets/Usernameeeeee/drone_test_2/commit/ac6f7dc8ffc0315265749d783d5b304001718dbb', commit_message='Upload dataset', commit_description='', oid='ac6f7dc8ffc0315265749d783d5b304001718dbb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Usernameeeeee/drone_test_2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Usernameeeeee/drone_test_2'), pr_revision=None, pr_num=None)

In [ ]:
# Add some to the dataset